# Snow Bear Fan Experience Analytics - Setup

This notebook sets up the complete Snow Bear analytics platform. Run each cell sequentially.

**⚠️ Prerequisites:**
1. Execute `setup.sql` to create database, role, warehouse, and stages
2. Download `basketball_fan_survey_data.csv.gz`, `snow_bear.py`, and `snowbear_fan_360.yaml`
3. Upload files to their respective stages in Snowsight
4. Import this notebook and run all cells

---


## 1. Load Data

Set up the database context and load the basketball fan survey data from the stage into the bronze layer table.

**What this does:**
- Switches to the Snow Bear role and warehouse
- Sets database and schema context
- Loads CSV data from stage using COPY INTO command


In [ ]:
-- Switch to Snow Bear role and set context
USE ROLE snow_bear_data_scientist;
USE WAREHOUSE snow_bear_wh;
USE DATABASE SNOW_BEAR_DB;
USE SCHEMA BRONZE_LAYER;


-- Load data from stage into bronze layer table
COPY INTO SNOW_BEAR_DB.BRONZE_LAYER.GENERATED_DATA_MAJOR_LEAGUE_BASKETBALL_STRUCTURED
FROM @SNOW_BEAR_DB.ANALYTICS.snow_bear_data_stage/basketball_fan_survey_data.csv.gz
FILE_FORMAT = SNOW_BEAR_DB.BRONZE_LAYER.csv_format
ON_ERROR = 'CONTINUE';


## 2. Create Gold Layer with Cortex AI Processing

Create the enhanced analytics table with Cortex AI sentiment analysis and theme extraction.

**What this does:**
- Creates QUALTRICS_SCORECARD table in the gold layer
- Applies SENTIMENT analysis to all comment fields
- Uses EXTRACT_ANSWER to generate theme summaries
- Sets up columns for themes, segments, and recommendations


In [ ]:
USE SCHEMA GOLD_LAYER;

-- Drop table if exists
DROP TABLE IF EXISTS SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD;

-- Create the complete AI-enhanced analytics table
CREATE OR REPLACE TABLE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
AS
SELECT DATEADD(DAY, UNIFORM(1, 365, RANDOM()), '2024-06-01') AS REVIEW_DATE,
       A.*,
       CAST(NULL AS INTEGER) as AGGREGATE_SCORE,       
       FOOD_OFFERING_COMMENT||' '||
       GAME_EXPERIENCE_COMMENT||' '||
       MERCHANDISE_OFFERING_COMMENT||' '||
       MERCHANDISE_PRICING_COMMENT||' '||
       OVERALL_EVENT_COMMENT||' '||
       PARKING_COMMENT||' '||
       SEAT_LOCATION_COMMENT   
       AS AGGREGATE_COMMENT,
       CAST(NULL AS FLOAT) AS AGGREGATE_SENTIMENT,
       CAST(NULL AS FLOAT) AS ALT_AGGREGATE_SENTIMENT,
       CAST(NULL AS FLOAT) AS AGGREGATE_SENTIMENT_SPREAD,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(FOOD_OFFERING_COMMENT), 2) AS FOOD_OFFERING_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(GAME_EXPERIENCE_COMMENT), 2) AS GAME_EXPERIENCE_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(MERCHANDISE_OFFERING_COMMENT), 2) AS MERCHANDISE_OFFERING_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(MERCHANDISE_PRICING_COMMENT), 2) AS MERCHANDISE_PRICING_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(OVERALL_EVENT_COMMENT), 2) AS OVERALL_EVENT_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(PARKING_COMMENT), 2) AS PARKING_SENTIMENT,   
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(SEAT_LOCATION_COMMENT), 2) AS SEAT_LOCATION_SENTIMENT,
       ROUND(SNOWFLAKE.CORTEX.SENTIMENT(STADIUM_COMMENT), 2) AS STADIUM_ACCESS_SENTIMENT,
       CAST(NULL AS VARCHAR(1000)) AS AGGREGATE_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(FOOD_OFFERING_COMMENT,'ASSIGN A THEME')[0]:answer::string AS FOOD_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(GAME_EXPERIENCE_COMMENT,'ASSIGN A THEME')[0]:answer::string AS GAME_EXPERIENCE_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(MERCHANDISE_OFFERING_COMMENT,'ASSIGN A THEME')[0]:answer::string AS MERCHANDISE_OFFERING_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(MERCHANDISE_PRICING_COMMENT,'ASSIGN A THEME')[0]:answer::string AS MERCHANDISE_PRICING_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(OVERALL_EVENT_COMMENT,'ASSIGN A THEME')[0]:answer::string  AS OVERALL_EVENT_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(PARKING_COMMENT,'ASSIGN A THEME')[0]:answer::string AS PARKING_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(SEAT_LOCATION_COMMENT,'ASSIGN A THEME')[0]:answer::string AS SEAT_LOCATION_SUMMARY,
       SNOWFLAKE.CORTEX.EXTRACT_ANSWER(STADIUM_COMMENT,'ASSIGN A THEME')[0]:answer::string AS STADIUM_ACCESS_SUMMARY,
       CAST(NULL AS VARCHAR(1000)) AS MAIN_THEME,
       CAST(NULL AS VARCHAR(1000)) AS SECONDARY_THEME,
       CAST(0 AS INTEGER) AS FOOD,
       CAST(0 AS INTEGER) AS PARKING,
       CAST(0 AS INTEGER) AS SEATING,    
       CAST(0 AS INTEGER) AS MERCHANDISE,      
       CAST(0 AS INTEGER) AS GAME,
       CAST(0 AS INTEGER) AS TICKET,
       CAST(0 AS INTEGER) AS NO_THEME,
       CAST(0 AS INTEGER) AS VIP,
       CAST(NULL as VARCHAR(1000)) AS SEGMENT,
       CAST(NULL as VARCHAR(1000)) AS SEGMENT_ALT,
       CAST(NULL AS VARCHAR(8000)) AS BUSINESS_RECOMMENDATION,       
       CAST(NULL AS VARCHAR(8000)) AS COMPLEX_RECOMMENDATION
FROM SNOW_BEAR_DB.BRONZE_LAYER.GENERATED_DATA_MAJOR_LEAGUE_BASKETBALL_STRUCTURED A;


## 3. Populate Theme and Summary Data

Populate the theme categories, create summary tables, and calculate aggregate sentiment scores.

**What this does:**
- Assigns main themes based on highest sentiment scores
- Creates EXTRACTED_THEMES_STRUCTURED table for app compatibility
- Calculates aggregate scores and sentiment analysis
- Updates sentiment spread calculations


In [ ]:
-- Populate MAIN_THEME column for Streamlit app compatibility
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD 
SET MAIN_THEME = 
    CASE 
        WHEN FOOD_OFFERING_SENTIMENT >= GREATEST(GAME_EXPERIENCE_SENTIMENT, PARKING_SENTIMENT, STADIUM_ACCESS_SENTIMENT, SEAT_LOCATION_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(FOOD_OFFERING_SENTIMENT) >= 0.3 THEN 'Food & Concessions'
        WHEN GAME_EXPERIENCE_SENTIMENT >= GREATEST(FOOD_OFFERING_SENTIMENT, PARKING_SENTIMENT, STADIUM_ACCESS_SENTIMENT, SEAT_LOCATION_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(GAME_EXPERIENCE_SENTIMENT) >= 0.3 THEN 'Game Experience'
        WHEN PARKING_SENTIMENT >= GREATEST(FOOD_OFFERING_SENTIMENT, GAME_EXPERIENCE_SENTIMENT, STADIUM_ACCESS_SENTIMENT, SEAT_LOCATION_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(PARKING_SENTIMENT) >= 0.3 THEN 'Parking'
        WHEN STADIUM_ACCESS_SENTIMENT >= GREATEST(FOOD_OFFERING_SENTIMENT, GAME_EXPERIENCE_SENTIMENT, PARKING_SENTIMENT, SEAT_LOCATION_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(STADIUM_ACCESS_SENTIMENT) >= 0.3 THEN 'Stadium Access'
        WHEN SEAT_LOCATION_SENTIMENT >= GREATEST(FOOD_OFFERING_SENTIMENT, GAME_EXPERIENCE_SENTIMENT, PARKING_SENTIMENT, STADIUM_ACCESS_SENTIMENT, OVERALL_EVENT_SENTIMENT) 
             AND ABS(SEAT_LOCATION_SENTIMENT) >= 0.3 THEN 'Seat Location'
        WHEN OVERALL_EVENT_SENTIMENT IS NOT NULL AND ABS(OVERALL_EVENT_SENTIMENT) >= 0.5 THEN 'Overall Event'
        ELSE 'Merchandise Quality'
    END;




-- Create EXTRACTED_THEMES_STRUCTURED table for Streamlit app compatibility
CREATE OR REPLACE TABLE SNOW_BEAR_DB.GOLD_LAYER.EXTRACTED_THEMES_STRUCTURED AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS THEME_NUMBER,
    MAIN_THEME,
    CASE 
        WHEN AVG(AGGREGATE_SENTIMENT) > 0.2 THEN 'Positive'
        WHEN AVG(AGGREGATE_SENTIMENT) < -0.2 THEN 'Negative'
        ELSE 'Neutral'
    END AS SENTIMENT_CATEGORY,
    'Theme extracted from fan feedback analysis using Cortex AI' AS THEME_DESCRIPTION,
    COUNT(*) AS RESPONSE_COUNT
FROM SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
WHERE MAIN_THEME IS NOT NULL
GROUP BY MAIN_THEME
ORDER BY COUNT(*) DESC;




-- Update aggregate scores and sentiment analysis
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
   SET AGGREGATE_SCORE = TRUNC((CASE WHEN FOOD_OFFERING_SCORE = 'N/A' THEN NULL ELSE FOOD_OFFERING_SCORE END+
                                CASE WHEN GAME_EXPERIENCE_SCORE = 'N/A' THEN NULL ELSE GAME_EXPERIENCE_SCORE END+
                                CASE WHEN MERCHANDISE_OFFERING_SCORE = 'N/A' THEN NULL ELSE MERCHANDISE_OFFERING_SCORE END +
                          CASE WHEN MERCHANDISE_PRICING_SCORE = 'N/A' THEN NULL ELSE MERCHANDISE_PRICING_SCORE END +
                          CASE WHEN OVERALL_EVENT_SCORE = 'N/A' THEN NULL ELSE OVERALL_EVENT_SCORE END +
                          CASE WHEN PARKING_SCORE = 'N/A' THEN NULL ELSE PARKING_SCORE END +
                          CASE WHEN SEAT_LOCATION_SCORE = 'N/A' THEN NULL ELSE SEAT_LOCATION_SCORE END +
                          CASE WHEN STADIUM_ACCESS_SCORE = 'N/A' THEN NULL ELSE STADIUM_ACCESS_SCORE END)/8),
       AGGREGATE_SENTIMENT = ROUND(SNOWFLAKE.CORTEX.SENTIMENT(AGGREGATE_COMMENT), 2),
       AGGREGATE_SUMMARY = SNOWFLAKE.CORTEX.EXTRACT_ANSWER(AGGREGATE_COMMENT,'ASSIGN A THEME')[0]:answer::string,
       ALT_AGGREGATE_SENTIMENT = (FOOD_OFFERING_SENTIMENT+GAME_EXPERIENCE_SENTIMENT+MERCHANDISE_OFFERING_SENTIMENT+
                                 MERCHANDISE_PRICING_SENTIMENT+OVERALL_EVENT_SENTIMENT+PARKING_SENTIMENT+
                                 SEAT_LOCATION_SENTIMENT+STADIUM_ACCESS_SENTIMENT)/8;
                                 
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
   SET AGGREGATE_SENTIMENT_SPREAD = ALT_AGGREGATE_SENTIMENT - AGGREGATE_SENTIMENT;
   



## 4. Create Cortex Search Service

Set up the Cortex Search service for semantic search capabilities across fan feedback.

**What this does:**
- Creates SNOWBEAR_SEARCH_ANALYSIS search service
- Indexes aggregate comments with relevant attributes
- Uses snowflake-arctic-embed-m-v1.5 embedding model
- Enables semantic search in the Streamlit app


In [ ]:
-- Create Cortex Search Service in the correct database context
USE DATABASE SNOW_BEAR_DB;
USE SCHEMA GOLD_LAYER;

CREATE OR REPLACE CORTEX SEARCH SERVICE SNOWBEAR_SEARCH_ANALYSIS
  ON AGGREGATE_COMMENT
  ATTRIBUTES AGGREGATE_SCORE,SEGMENT, SEGMENT_ALT, MAIN_THEME, SECONDARY_THEME,
        PARKING_SCORE,SEAT_LOCATION_SCORE,
        OVERALL_EVENT_SCORE,MERCHANDISE_PRICING_SCORE,
        MERCHANDISE_OFFERING_SCORE,GAME_EXPERIENCE_SCORE,
        FOOD_OFFERING_SCORE,REVIEW_DATE,ID
  WAREHOUSE = snow_bear_wh
  TARGET_LAG = '1 days'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-m-v1.5'
  INITIALIZE = ON_CREATE 
  COMMENT = 'CORTEX SEARCH SERVICE FOR SNOW BEAR FAN EXPERIENCE ANALYSIS' 
  AS (
    SELECT
		AGGREGATE_COMMENT,AGGREGATE_SCORE,
        SEGMENT, SEGMENT_ALT, MAIN_THEME, SECONDARY_THEME,
        PARKING_SCORE,SEAT_LOCATION_SCORE,
        OVERALL_EVENT_SCORE,MERCHANDISE_PRICING_SCORE,
        MERCHANDISE_OFFERING_SCORE,GAME_EXPERIENCE_SCORE,
        FOOD_OFFERING_SCORE,REVIEW_DATE,ID
	FROM "SNOW_BEAR_DB"."GOLD_LAYER"."QUALTRICS_SCORECARD");


## 5. Generate Business Recommendations

Use Cortex AI to generate personalized business recommendations based on fan feedback and sentiment.

**What this does:**
- Generates targeted recommendations using CORTEX.COMPLETE
- Creates different recommendation types based on sentiment scores
- Provides actionable business insights for each fan profile
- Populates both simple and complex recommendation fields


In [ ]:
-- Generate business recommendations using Cortex AI
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
SET BUSINESS_RECOMMENDATION = 
    CASE 
        WHEN AGGREGATE_SCORE <= 2 THEN 
            SNOWFLAKE.CORTEX.COMPLETE(
                'snowflake-arctic',
                CONCAT('Based on this negative fan feedback: "', AGGREGATE_COMMENT, '", provide a specific business recommendation to improve the fan experience. Focus on actionable steps for: ', MAIN_THEME, '. Keep response under 100 words.')
            )
        WHEN AGGREGATE_SCORE >= 4 THEN 
            SNOWFLAKE.CORTEX.COMPLETE(
                'snowflake-arctic', 
                CONCAT('Based on this positive fan feedback: "', AGGREGATE_COMMENT, '", provide a business recommendation on how to maintain and expand on these successful aspects of: ', MAIN_THEME, '. Keep response under 100 words.')
            )
        ELSE 
            CONCAT('Focus on improving ', MAIN_THEME, ' experience based on mixed feedback. Consider surveying fans for specific improvement ideas.')
    END
WHERE BUSINESS_RECOMMENDATION IS NULL AND AGGREGATE_COMMENT IS NOT NULL;

-- Update COMPLEX_RECOMMENDATION for premium experience insights
UPDATE SNOW_BEAR_DB.GOLD_LAYER.QUALTRICS_SCORECARD
SET COMPLEX_RECOMMENDATION = 
    SNOWFLAKE.CORTEX.COMPLETE(
        'snowflake-arctic',
        CONCAT('Fan Profile: Score=', AGGREGATE_SCORE, ', Segment=', SEGMENT, ', Theme=', MAIN_THEME, '. Comment: "', AGGREGATE_COMMENT, '". Provide a comprehensive business strategy recommendation for this fan profile. Include retention strategy, upsell opportunities, and experience personalization. Limit to 150 words.')
    )
WHERE COMPLEX_RECOMMENDATION IS NULL AND SEGMENT IS NOT NULL AND AGGREGATE_COMMENT IS NOT NULL;


## 6. Create Streamlit Application

Deploy the interactive analytics dashboard from the uploaded Python file.

**What this does:**
- Creates the Snow Bear Fan Analytics Streamlit app
- Points to the uploaded snow_bear.py file in the stage
- Grants usage permissions to the Snow Bear role
- Makes the dashboard accessible via Projects → Streamlit


In [ ]:
-- Create the Streamlit app from the uploaded file
USE DATABASE SNOW_BEAR_DB;
USE SCHEMA ANALYTICS;

CREATE OR REPLACE STREAMLIT "Snow Bear Fan Analytics"
    ROOT_LOCATION = '@SNOW_BEAR_DB.ANALYTICS.SNOW_BEAR_DATA_STAGE'
    MAIN_FILE = 'snow_bear.py'
    QUERY_WAREHOUSE = 'SNOW_BEAR_WH'
    COMMENT = 'Snow Bear Fan Experience Analytics Dashboard';

-- Grant usage to the Snow Bear role
GRANT USAGE ON STREAMLIT "Snow Bear Fan Analytics" TO ROLE SNOW_BEAR_DATA_SCIENTIST;


## ✅ Setup Complete!

Navigate to **Projects → Streamlit → Snow Bear Fan Analytics** to access your dashboard.
